# **IMPORT LIBS**

## Import

Import standard libraries.

In [1]:
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.recommenders import CML
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt  
import pandas as pd
import os
import sys

Import our functions.

In [2]:
# Adjusts path to include the utilities.py file
sys.path.append('../')
# Imports it
from helper import *

Set training flag.

In [4]:
# Flag to retrain the model (in this notebook should always be False)
REPEAT_TRAINING = False

# **INITIALIZATION**

## Init

Initialize notebook parameters.

In [ ]:
# Set the seed for reproducibility
seed = 2384795
np.random.seed(seed=seed)

Initialize paths.

In [ ]:
# Preparing folder for output data
output_name = f"./generated_data/"
if os.path.exists(output_name) == False:
    os.makedirs(output_name)

# **LOAD MODEL AND DATA**

In this section we will just be using the code provided by the authors of the paper.

## Init

Prepare names and paths.

In [6]:
# Initialize metadata and paths
MODEL_CLASS = CML
MODEL_PREFIX = "cml"
DATASET_NAME = "yahoo"
OUTPUT_FOLDER = output_name
OUTPUT_PATH = OUTPUT_FOLDER + MODEL_PREFIX + "-" + DATASET_NAME + "/"
OUTPUT_PREFIX = str(OUTPUT_PATH) + str(MODEL_PREFIX) + "-" + str(DATASET_NAME)
if os.path.exists(OUTPUT_PATH) == False:
    os.makedirs(OUTPUT_PATH)

## Load dataset and model

Import data from output files.

In [7]:
# Init dictionary for 
raw_data = dict()
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001

# Load train and test data from files
raw_data['train_data'] = np.load(output_name + "training_arr.npy")
raw_data['test_data_pos_biased'] = np.load(output_name + "biased-test_arr_pos.npy")
raw_data['test_data_neg_biased'] = np.load(output_name + "biased-test_arr_neg.npy")
raw_data['test_data_pos_unbiased'] = np.load(output_name + "unbiased-test_arr_pos.npy")
raw_data['test_data_neg_unbiased'] = np.load(output_name + "unbiased-test_arr_neg.npy")

# Form the datasets
train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos_biased = ImplicitDataset(raw_data['test_data_pos_biased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg_biased = ImplicitDataset(raw_data['test_data_neg_biased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_pos_unbiased = ImplicitDataset(raw_data['test_data_pos_unbiased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg_unbiased = ImplicitDataset(raw_data['test_data_neg_unbiased'], raw_data['max_user'], raw_data['max_item'])

Load model.

In [ ]:
# Prevent tensorflow from using cached embeddings and set seed
import tensorflow as tf
tf.compat.v1.reset_default_graph()
tf.set_random_seed(seed)

# Set parameters to define the model
batch_size = 8000
test_batch_size = 1000

# Define the model
model = MODEL_CLASS(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(), dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size, train_dataset=train_dataset, model=model, sampler=sampler, eval_save_prefix=OUTPUT_PATH + DATASET_NAME, item_serving_size=500)
auc_evaluator = AUC()

# Load the model
model.load(OUTPUT_PATH)

# Set further parameters
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos_biased, test_dataset_neg_biased])
model_trainer._sample_negatives(seed=10)

# **EVALUATION**

In this section we will just be using the code provided by the authors of the paper.

## Biased Evaluation

Compute recommendations with biased testset.

In [ ]:
# Set output path for pickles files
model_trainer._eval_save_prefix = OUTPUT_PREFIX + "-test-pos-biased"
# Evaluate
model_trainer._evaluate_partial(test_dataset_pos_biased)

# Set output path for pickles files
model_trainer._eval_save_prefix = OUTPUT_PREFIX +  "-test-neg-biased"
# Evaluate
model_trainer._evaluate_partial(test_dataset_neg_biased)

## Unbiased Evaluation

Compute recommendations with biased testset.

In [ ]:
# Set output path for pickles files
model_trainer._eval_save_prefix = OUTPUT_PREFIX + "-test-pos-unbiased"
# Evaluate
model_trainer._evaluate_partial(test_dataset_pos_unbiased)

# Set output path for pickles files
model_trainer._eval_save_prefix = OUTPUT_PREFIX +  "-test-neg-unbiased"
# Evaluate
model_trainer._evaluate_partial(test_dataset_neg_unbiased)

# **COMPUTE RESULTS**

## Preprocessing

Here we first preprocess the propensities from the training set data.

In [12]:
# Preprocess the propensities
propensities = calculate_propensities(raw_data['max_user'], raw_data['max_item'], output_name+"training_arr.npy", normalize=True)

## Compute rivals metrics

Now set the gamma values to be considered.

In [ ]:
# Choose values for gamma
GAMMAS = [1.5, 2, 2.5, 3]

Compute AOA and IPS metrics for both biased and unbiased datasets.

In [14]:
# Init biased and unbiased results dictionaries
biased_results = dict()
unbiased_results = dict()

# Compute AOA results
biased_results["AOA"] = aoa(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", output_name+"training_arr.npy", K=30)
unbiased_results["AOA"] = aoa(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", output_name+"training_arr.npy", K=1)

# Compute IPS results
for gamma in GAMMAS:
    key = "UB_" + str(gamma).replace(".","")
    biased_results[key] = eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", output_name+"training_arr.npy", propensities[gamma], K=30)
    unbiased_results[key] = eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", output_name+"training_arr.npy", propensities[gamma], K=1)

## Compute Stratified

Get array of possible values for the number of subsets in the partition.

In [ ]:
# Get number of items
num_items = raw_data['max_item'] - 1

# Set the number of possible values to be considered
n_p = 300

# Get array with the first num_items natural numbers 
nums = np.arange(1, num_items+1)

# Choose n_p random values
partitions = np.random.choice(nums, n_p, replace=False)

Compute the partition p which minimizes the sum of Bias and Conc.

In [ ]:
# Value of gamma to use for minimization
opt_gamma = 1.5

# Keys prefix in the dictionaries + gamma value
key = "STRATIFIED_" + str(opt_gamma).replace(".","")

# Init dictionaries to store results for biased and unbiased testset
unbiased_results[key] = {}
biased_results[key] = {}

# Random init for best partition
best_partition = np.random.choice(nums, 1)[0]
# Init infinity as min score
best_score = float('inf')

# To plot the results init histories
history_objective = np.full(num_items, np.inf)
history_mae_auc = np.full(num_items, np.inf)
history_mae_recall = np.full(num_items, np.inf)
history_bound = np.full(num_items, np.inf)

# For each value in the partitions array (n_p random values from 1 to num_items)
for p in tqdm(partitions):

    # Fetch stratified results
    temp_unbiased = stratified(OUTPUT_PREFIX + "-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX + "-test-neg-unbiased_evaluate_partial.pickle", output_name + "training_arr.npy", propensities[opt_gamma], K=1, partition=p)
    temp_biased = stratified(OUTPUT_PREFIX + "-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX + "-test-neg-biased_evaluate_partial.pickle", output_name + "training_arr.npy", propensities[opt_gamma], K=30, partition=p)
      
    # Computing bound score
    bound_score = temp_unbiased['bias'] + temp_unbiased['concentration'] + temp_biased['bias'] + temp_biased['concentration']
    # Compute maes to plot the history
    mae_score_auc = abs(temp_unbiased['auc'] - temp_biased['auc'])   
    mae_score_recall = abs(temp_unbiased['recall'] - temp_biased['recall'])
    
    # Set objective (to switch between the bound and the MAE to check whether the bound is a good metric)
    objective = bound_score 
    
    # Storing historical values
    history_mae_auc[p-1] = mae_score_auc  # Store the mae using auc
    history_mae_recall[p-1] = mae_score_recall  # Store the mae using recall
    history_bound[p-1] = bound_score  # Store the bound score

    # Update the best_partition and best_score if the current partition's bound is lower
    if objective < best_score:
        best_score = objective
        best_partition = p

# Print the best partition and the combined score
print(f"Best partition: {best_partition} with combined score: {best_score}")

Plot MAE with AUC over each considered partition.

In [ ]:
# Get p values (only the ones which were considered)
x = np.where(history_mae_auc != np.inf)
# Get MAE values with AUC
y = history_mae_auc[history_mae_auc != np.inf]
 
# Plot
plt.title("MAE using auc") 
plt.xlabel("NUM PARTITIONS") 
plt.ylabel("MAE using auc") 
plt.scatter(x, y, color ="red") 
plt.show()

Plot MAE with Recall over each considered partition.

In [ ]:
# Get p values (only the ones which were considered)
x = np.where(history_mae_recall != np.inf)
# Get MAE values with recall
y = history_mae_recall[history_mae_recall != np.inf]
 
# Plot
plt.title("MAE using recall") 
plt.xlabel("NUM PARTITIONS") 
plt.ylabel("MAE using recall") 
plt.scatter(x, y, color ="red") 
plt.show()

Plot the bound over each considered partition.

In [ ]:
# Get p values (only the ones which were considered)
x = np.where(history_bound != np.inf)
# Get the bound values
y = history_bound[history_bound != np.inf]
 
# Plot
plt.title("Bound values") 
plt.xlabel("NUM PARTITIONS") 
plt.ylabel("Bound value") 
plt.scatter(x, y, color ="red") 
plt.show()

Now finally compute the evaluation with the Stratified estimator.

In [24]:
# Compute Stratified results
for gamma in GAMMAS:
    key = "STRATIFIED_" + str(gamma).replace(".","")
    unbiased_results[key] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", output_name+"training_arr.npy", propensities[gamma], K=1, partition=best_partition)
    biased_results[key] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", output_name+"training_arr.npy", propensities[gamma], K=30, partition=best_partition)

This version builds a partition evenly distributing the items instead of the propensities.

In [25]:
# Compute Stratified results with the second version
for gamma in GAMMAS:
    key = "STRATIFIED_v2_" + str(gamma).replace(".","")
    unbiased_results[key] = stratified_2(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", output_name+"training_arr.npy", propensities[gamma], K=1, partition=best_partition)
    biased_results[key] = stratified_2(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", output_name+"training_arr.npy", propensities[gamma], K=30, partition=best_partition)


Prepare table for results.

In [ ]:
# Set number of columns as number of estimators which were used
columns = len(biased_results.keys())

# Determine the maximum number (number of used metrics)
rows = max(max(len(biased_results[key].keys()) for key in biased_results.keys()), max(len(unbiased_results[key].keys()) for key in unbiased_results.keys()))

# Init matrix to store results
results_array = np.zeros((rows,columns))

# Get the names of the rows
list_biased_res = list(biased_results.keys())

Fill the table with the MAE results and get the DataFrame object.

In [28]:
# For each evaluator
for i in range(len(list_biased_res)):
    key = list_biased_res[i]

    # For each metric
    for j in range(len(list(biased_results[key].keys()))):
        key_2 = list(biased_results[key].keys())[j]

        # Compute MAE between biased and unbiased results
        results_array[j][i] = abs(biased_results[key][key_2] - unbiased_results[key][key_2])

# Make it a DataFrame
mae_df = pd.DataFrame(columns=list(biased_results.keys()), data=results_array)
metric_values = list(biased_results[list(biased_results.keys())[0]].keys())
mae_df.insert(0, "metric", metric_values)

# **RESULTS**

## Visualize

Remembering that...

In [ ]:
# Print the results
print("Minimization was done for gamma = ", opt_gamma, ". The best number of partitions: ", best_partition)

Visualize.

In [ ]:
# Visualize
mae_df.head()